# Mini projet 3: Les profils PSSM

Le but de ce projet est de construire un profil pou rla famille de séquences WW et de comparer votre résultat avec le profil qu'on peut trouver sur le site web PFAM pour la même famille. Comme avant, n'oubliez pas de bien expliquez comment vous avez trouvé votre solution dans votre document Jupyter.

## Données

J'ai choisis d'aligner les séquences de la famille _WW_ d'utiliser les outils [CLUSTAL Omega](http://www.ebi.ac.uk/Tools/msa/clustalo/) et [MUSCLE](http://www.ebi.ac.uk/Tools/msa/muscle/). 

## Implémentation

Je réutilise le parser que j'ai utilisez dans mes précédents projets pour lire des fichiers _.fasta_

In [71]:
VALID_AMINO = "ARNDCQEGHILKMFPSTWYVBZX"

In [72]:
class Sequence():
    """
    ADT séquence qui représente une séquence d’acides aminés
    et tous les opérations qu’on peut exécuter sur une séquence.
    """
    def __init__(self, sequence):
        self.current = 0
        self.sequence = sequence
    
    def __repr__(self):
        return str(self.sequence)
    
    def __len__(self):
        return len(self.sequence)
    
    def __iter__(self):
        self.current = 0
        return self
    
    def next(self):
        if self.current > len(self.sequence):
            raise StopIteration
        else:
            self.current += 1
            return self.sequence[self.current - 1]
    
    def __getitem__(self, index):
        """
        @desc: Permet d'interpreter la classe comme une "String".
        
        @param{index}: Index de la lettre qu'on veut consulter.
        """
        return self.sequence[index]

    @staticmethod
    def load(filename):
        """
        """
        with open(filename) as f:
            seq = []
            for line in f:
                if line[0] == ">":
                    if seq:
                        yield Sequence(''.join(seq))
                    seq = [] 
                else:
                    seq.append(line.strip())
                    
            if (seq):
                yield Sequence(''.join(seq))

J'importe les résultats des alignements executé à l'aide des outils _CLUSTAL Omega_ et _MUSCLE_ dans le notebook.

In [73]:
clustal = [seq for seq in Sequence.load('./msaresults-CLUSTAL.fasta')]
muscle = [seq for seq in Sequence.load('./msaresults-MUSCLE.fasta')]

In [81]:
# les proba sont hard-coder | trouver sur swissprot
PROBABILITY_AMINO = {
    "A": 0.0826,   "Q": 0.0393,   "L": 0.0966,   "S": 0.0657,
    "R": 0.0553,   "E": 0.0674,   "K": 0.0583,   "T": 0.0534,
    "N": 0.0405,   "G": 0.0708,   "M": 0.0241,   "W": 0.0109,
    "D": 0.0546,   "H": 0.0227,   "F": 0.0386,   "Y": 0.0292,
    "C": 0.0137,   "I": 0.0595,   "P": 0.0471,   "V": 0.0687,
    "B": 0.0000,   "Z": 0.0000,  "X": 0.0000
}

In [82]:
from math import log, sqrt


class Profil(object):
    """ 
    Classe permetant de créer un profile 
    """
    def __init__(self, seqCollection):
        self._seqCollection = seqCollection
        self._validAmino = VALID_AMINO
        self._seqSize = len(self._seqCollection[0])

        self._alpha = len(self._seqCollection) - 1
        self._beta = sqrt(len(self._seqCollection))

        # initialisation du profile
        # pour des raisons de performance/facilité. Ce sera une liste
        # de dico dont les clés sont les acides aminés
        self._profile = [None for i in range(self._seqSize)]
        for iSeq in range(self._seqSize):
            self._profile[iSeq] = dict()
            for amino in self._validAmino:
                self._profile[iSeq][amino] = 0

        self._consensusSeq = ""

        self._make()

    def _addFrequency(self):
        """
        Ajoute la fréquence pour chaque acide aminé au profile 
        """
        for seq in self._seqCollection:
            for i in range(self._seqSize):
                if seq[i] != "-":
                    self._profile[i][seq[i]] += (1 / len(self._seqCollection))

    def _addQ(self):
        """
        Ajoute des pseudo count au profile
        """
        for i in range(self._seqSize):
            for amino in self._validAmino:
                # ajout des pseudo count
                self._profile[i][amino] = (self._alpha * self._profile[i][amino]) + (self._beta * PROBABILITY_AMINO[amino])
                self._profile[i][amino] /= (self._alpha + self._beta)

    def _addScore(self):
        """ 
        Ajoute le score (log finale) au profile
        """
        for i in range(self._seqSize):
            for amino in self._validAmino:
                if PROBABILITY_AMINO[amino] == 0 and self._profile[i][amino] == 0:
                    # limite log tendant => 0 ==  (log(0/0)) == -inf
                    # car une proba réellement nul n'est pas possible => log(0)
                    self._profile[i][amino] = float("-inf")
                elif PROBABILITY_AMINO[amino] != 0: #log(nbr-reel)
                    score = self._profile[i][amino] / PROBABILITY_AMINO[amino]
                    self._profile[i][amino] = log(score, 10)
                else: # log(inf)
                    self._profile[i][amino] = float("inf")

    def _makeConsensus(self):
        """ Crée la séquence consensus"""
        self._consensusSeq = ""
        for i in range(self._seqSize):
            # on ajoute a chaque fois le meilleur amino au consensus
            bestAmino = self._validAmino[0]
            for amino in self._validAmino:
                if self._profile[i][amino] > self._profile[i][bestAmino]:
                    bestAmino = amino
                    
            self._consensusSeq += bestAmino

    def _make(self):
        """
        Cette méthode ajoute les différentes étapes au profil et crée le consensus
        """
        self._addFrequency()
        self._addQ()
        self._addScore()
        
        self._makeConsensus()

    def getConsensus(self):
        """
        Renvoi la séquence consensus
        """
        return self._consensusSeq

    def getProfile(self):
        """
        Renvoi le profile 
        """
        return self._profile

    def profiletoFile(self, filename):
        """ 
        Cette méthode sauvegarde le profile dans un fichier
        """
        f = open(filename, "w")
        # Première ligne : numéro des colonnes
        f.write('   ')
        for i in range(self._seqSize):
            f.write(str(i) + "      ")
        f.write("\n")
        # Ensuite pour chaque acide aminé, on ecrit les données de colonnes
        for a in self._validAmino:
            f.write(a + "   ")
            for i in range(self._seqSize):
                f.write(str(round(self._profile[i][a], 3)) + "  ")
            f.write("\n")
        f.close()

    def consensusToFile(self, filename):
        """
        Cette méthode sauvegarde le consensus dans un fichier
        """
        f = open(filename, "w")
        f.write(self._consensusSeq)
        f.close()

### L'algorithme

L'algorithme se déroule en plusieurs étapes pour calculer le PSSM

La première est de calculer la fréquence d'apparition de chaque acide aminé sur l'ensemble sur chaque séquence.

$f_{u,b} = \frac{n_{u,b}}{N_{seq}}$ est la fréquence d'apparition d'un acide aminé sur une séquence avec $n_{u,b}$ qui est le nombre de fois qu'apparait un acide aminé dans une séquence et $N_{seq}$ qui est le nombre de séquence.

Ce calcul est défini dans la fonction `def _addFrequency(self):`.

La deuxième étape est de calculer la probabilitée de trouver un acide aminé dans une séquence.

$$\alpha = \sqrt{N_{seq}}$$
$$N'_{seq} = nombre\ de\ séquence\ sans\ "gap"\ à\ cette\ position$$
$$\beta = N'_{seq}$$

$p_a$ est la probabilité d'obtenir un acide aminé trouvé sur _SWISSPROT_ que j'ai traduit comme suit dans mon implémentation:

```
PROBABILITY_AMINO = {
    "A": 0.0826,   "Q": 0.0393,   "L": 0.0966,   "S": 0.0657,
    "R": 0.0553,   "E": 0.0674,   "K": 0.0583,   "T": 0.0534,
    "N": 0.0405,   "G": 0.0708,   "M": 0.0241,   "W": 0.0109,
    "D": 0.0546,   "H": 0.0227,   "F": 0.0386,   "Y": 0.0292,
    "C": 0.0137,   "I": 0.0595,   "P": 0.0471,   "V": 0.0687,
    "B": 0.0000,   "Z": 0.0000,  "X": 0.0000
}
```

La probabilitée est donc:

$$q_{u,a} = \frac{\alpha f_{u,a} + \beta p_a}{\alpha + \beta}$$

La probabilitée est calculée dans la méthode `def _addFrequency(self):`

Le score final est calculé avec la formule suivante:

$$m_{u,a} = log \frac{q_{u,a}}{p_a}$$

In [83]:
c = Profil(clustal)
m = Profil(muscle)

In [84]:
m.profiletoFile('test')